In [1]:
from collections import deque
import pandas as pd
from IPython.display import display

# Define the time quantum
time_quantum = 1
# Set pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Process data: ProcessID, ArrivalTime, BurstTime
processes = [("A", 4, 3), ("B", 3, 7), ("C", 1, 4), ("D", 5, 2), ("E", 0, 2), ("F", 2, 4)]

# Sort processes by their arrival time
processes.sort(key=lambda x: x[1])

# Initialize variables
queue = deque()
current_time = 0
timeline = []
waiting_time = {process[0]: 0 for process in processes}
turnaround_time = {process[0]: 0 for process in processes}
remaining_burst_time = {process[0]: process[2] for process in processes}
process_arrived = {process[0]: False for process in processes}

# Start the round-robin scheduling
while any(remaining_burst_time[p] > 0 for p in remaining_burst_time):
    # Add processes to the queue that have arrived
    for process in processes:
        if process[1] <= current_time and not process_arrived[process[0]]:
            queue.append(process[0])
            process_arrived[process[0]] = True

    if queue:
        current_process = queue.popleft()
        exec_time = min(time_quantum, remaining_burst_time[current_process])
        remaining_burst_time[current_process] -= exec_time
        timeline.append((current_process, current_time, current_time + exec_time))
        current_time += exec_time

        if remaining_burst_time[current_process] > 0:
            queue.append(current_process)
        else:
            turnaround_time[current_process] = current_time - [p[1] for p in processes if p[0] == current_process][0]
    else:
        current_time += 1

# Calculate average waiting and turnaround times
for process in processes:
    process_id = process[0]
    arrival_time = process[1]
    waiting_time[process_id] = turnaround_time[process_id] - process[2]

average_waiting_time = sum(waiting_time.values()) / len(waiting_time)
average_turnaround_time = sum(turnaround_time.values()) / len(turnaround_time)

# Create a DataFrame for better visualization
df_timeline = pd.DataFrame(timeline, columns=["ProcessID", "StartTime", "EndTime"])
df_waiting_turnaround = pd.DataFrame(list(waiting_time.items()), columns=["ProcessID", "WaitingTime"])
df_waiting_turnaround["TurnaroundTime"] = df_waiting_turnaround["ProcessID"].map(turnaround_time)

# Define the maximum time units for the table
max_time_units = max(timeline, key=lambda x: x[2])[2]

# Initialize the table with empty strings
execution_table = {process_id: [" " for _ in range(max_time_units)] for process_id in waiting_time}

# Fill the table with 'X' where processes are executing
for process_id, start, end in timeline:
    for time_unit in range(start, end):
        execution_table[process_id][time_unit] = "X"

# Convert the table to a DataFrame for better visualization
df_execution_table = pd.DataFrame(execution_table)
df_execution_table.index += 1  # Adjusting index to start from 1
df_execution_table.index.name = "_"
df_execution_table.columns.name = "ProcessID"

# Transpose the DataFrame to swap rows and columns
df_execution_transposed = df_execution_table.transpose()

# Reordering the columns to display processes in the order A, B, C, D, E, F
ordered_processes = ['A', 'B', 'C', 'D', 'E', 'F']
df_execution_ordered = df_execution_transposed.loc[ordered_processes]

# Styling and displaying the DataFrame directly
styled_df = df_execution_ordered.style.map(
    lambda x: 'background-color: lightgreen' if x == 'X' else 'background-color: white'
)
display(df_execution_ordered)

_,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
ProcessID,,,,,,,,,,,,,,,,,,,,,,
A,,,,,,,,X,,,,,X,,,,,X,,,,
B,,,,,,X,,,,,X,,,,,X,,,X,X,X,X
C,,,X,,X,,,,X,,,,,X,,,,,,,,
D,,,,,,,,,,X,,,,,X,,,,,,,
E,X,X,,,,,,,,,,,,,,,,,,,,
F,,,,X,,,X,,,,,X,,,,,X,,,,,


Time 1: Process E
Time 2: Process E
Time 3: Process C
Time 4: Process F
Time 5: Process C
Time 6: Process B
Time 7: Process F
Time 8: Process A
Time 9: Process C
Time 10: Process D
Time 11: Process B
Time 12: Process F
Time 13: Process A
Time 14: Process C
Time 15: Process D
Time 16: Process B
Time 17: Process F
Time 18: Process A
Time 19: Process B
Time 20: Process B
Time 21: Process B
Time 22: Process B
